In [2]:
import time
import os
import uuid
from pathlib import Path

import numpy as np
import pandas as pd
import requests
import torch
import wandb
from pytorch_tabular import TabularModel
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig
from pytorch_tabular.models import GatedAdditiveTreeEnsembleConfig
import torch
from thop import profile
from thop import clever_format

DATA_DIR = Path("data")
EXP_RES = "data/tuning_results_best_rows.parquet"

In [7]:
config = {
        "dataset": "heloc",
        "batch_size": 512,
        "max_epochs": 100,
        "early_stopping_patience": 10,
        "optimizer": "AdamW",
        "wandb": True,
    }

In [26]:
def simulate_training_best_params(config):
    # "model__use_checkpoints": True} #TODO
    data_path = DATA_DIR / config["dataset"]
    exp_res = pd.read_parquet(EXP_RES)
    d_config_files = list(data_path.glob("*config*"))
    d_config = np.load(d_config_files[0], allow_pickle=True).item()
    task = "classification" if d_config["regression"] == 0 else "regression"
    n_folds = len(d_config_files)
    config["n_folds"] = n_folds
    if d_config.get("data__categorical", 0) == 1:
        categorical_indicator = np.load(
            data_path / "categorical_indicator_fold_0.npy", allow_pickle=True
        )
        feat_names = [f"feature_{i}" for i in range(categorical_indicator.shape[0])]
        cat_col_names = [
            feat_names[i]
            for i in range(len(feat_names))
            if categorical_indicator[i] == 1
        ]
        num_col_names = [
            feat_names[i]
            for i in range(len(feat_names))
            if categorical_indicator[i] == 0
        ]
    else:
        n_features = np.load(
            data_path / "x_test_fold_0.npy", allow_pickle=True
        ).shape[1]
        cat_col_names = None
        num_col_names = [f"feature_{i}" for i in range(n_features)]
    best_params = (
        exp_res.xs(config["dataset"], level="data__keyword")
        .xs("GFLU", level="model_name")["params"]
        .item()
    )
    model_params = {
        k.replace("params_", ""): v
        for k, v in best_params.items()
        if ("optimizer" not in k)
    }
    model_params["gflu_stages"] = int(model_params["gflu_stages"])
    model_params['num_trees'] = 0
    model_params["feature_mask_function"] = "t-softmax"
    model_params["head_config"] = {"layers": "32-16"}
    # model_params["learnable_sparsity"] = True
    # model_params["gflu_dropout"] = 0.0 
    # model_params["gflu_feature_init_sparsity"] = 0.1621935267229608
    # model_params["learning_rate"] = 0.001 # just for  mercedes
    weight_decay = best_params["params_optimizer_config__weight_decay"]
    print(f"Using Data from: {data_path} for task: {task} with {n_folds} folds")
    data_config = DataConfig(
        target=["target"],
        continuous_cols=num_col_names,
        categorical_cols=cat_col_names or [],
        normalize_continuous_features=True,
    )
    trainer_config = TrainerConfig(
        auto_lr_find=False,  # Runs the LRFinder to automatically derive a learning rate
        batch_size=config["batch_size"],
        max_epochs=config["max_epochs"],
        early_stopping="valid_loss",
        early_stopping_mode="min",  # Set the mode as min because for val_loss, lower is better
        early_stopping_patience=config[
            "early_stopping_patience"
        ],  # No. of epochs of degradation training will wait before terminating
        # early_stopping_min_delta=0.0001,  # Minimum delta for improvement in val_loss
        checkpoints="valid_loss",
        fast_dev_run=True,
        load_best=True,  # After training, load the best checkpoint
        # progress_bar="none",  # Turning off Progress bar
        # trainer_kwargs=dict(enable_model_summary=False),  # Turning off model summary
    )
    optimizer_config = OptimizerConfig(
        optimizer=config["optimizer"], optimizer_params={"weight_decay": weight_decay}
    )
    model_config = GatedAdditiveTreeEnsembleConfig(
        task=task,
        metrics=["r2_score"] if task == "regression" else None,
        metrics_prob_input=[False] if task == "regression" else None,
        **model_params,
    )
    metric_name = "test_r2_score" if task == "regression" else "test_accuracy"
    model_id = uuid.uuid4().hex
    name = f"{config['dataset']}_{model_id}"
    print(config)
    # print(config)
    test_scores = []
    times = []
    for fold in range(n_folds):
        start_time = time.time()
        x_train = np.load(
            data_path / f"x_train_fold_{fold}.npy", allow_pickle=True
        )
        y_train = np.load(
            data_path / f"y_train_fold_{fold}.npy", allow_pickle=True
        ).reshape(-1, 1)
        x_val = np.load(data_path / f"x_val_fold_{fold}.npy", allow_pickle=True)
        y_val = np.load(
            data_path / f"y_val_fold_{fold}.npy", allow_pickle=True
        ).reshape(-1, 1)
        x_test = np.load(
            data_path / f"x_train_fold_{fold}.npy", allow_pickle=True
        )
        y_test = np.load(
            data_path / f"y_train_fold_{fold}.npy", allow_pickle=True
        ).reshape(-1, 1)
        # combine x and y into a dataframe
        train = pd.DataFrame(
            np.concatenate([x_train, y_train], axis=1),
            columns=[f"feature_{i}" for i in range(x_train.shape[1])]
            + ["target"],
        )
        val = pd.DataFrame(
            np.concatenate([x_val, y_val], axis=1),
            columns=[f"feature_{i}" for i in range(x_val.shape[1])] + ["target"],
        )
        test = pd.DataFrame(
            np.concatenate([x_test, y_test], axis=1),
            columns=[f"feature_{i}" for i in range(x_test.shape[1])] + ["target"],
        )
        # Initialize the tabular model
        tabular_model = TabularModel(
            data_config=data_config,
            model_config=model_config,
            optimizer_config=optimizer_config,
            trainer_config=trainer_config,
        )
        try:
            # Fit the model
            if name.startswith("Ailerons"):
                # Need special transformation to target to avoid gradient underflow
                tabular_model.fit(
                    train=train,
                    validation=val,
                    target_transform=(lambda x: x * 1000, lambda x: x / 1000),
                )
            else:
                tabular_model.fit(
                    train=train,
                    validation=val,
                )
            end_time = time.time()
        except RuntimeError as e:
                    # print(e)
                    # gc.collect()
                    # torch.cuda.empty_cache()
                    raise e
        break
    return tabular_model

In [28]:
datasets = [
    "default-of-credit-card-clients",
    "heloc",
    "eye_movements",
    "Higgs",
    "pol",
    "albert",
    "road-safety",
    "MiniBooNE",
    "covertype",
    "jannis",
    "Bioresponse",
    "cpu_act",
    "Ailerons",
    "yprop_4_1",
    "superconduct",
    "Allstate_Claims_Severity",
    "topo_2_1",
    "Mercedes_Benz_Greener_Manufacturing"
    ]
res_l = []
for col in datasets:
    config["dataset"] = col
    tabular_model = simulate_training_best_params(config)
    mdl = tabular_model.model
    mdl.eval()
    tabular_model.datamodule.batch_size = 1
    x = next(iter(tabular_model.datamodule.train_dataloader()))
    macs, params = profile(mdl, inputs=(x,))
    macs_h, params_h = clever_format([macs, params], "%.3f")
    res = {
        "dataset": col,
        "model" : "GANDALF",
        "macs": macs,
        "params": params,
        "macs_h": macs_h,
        "params_h": params_h,
    }
    res_l.append(res)

2023-06-25 18:21:59,936 - {pytorch_tabular.tabular_model:105} - INFO - Experiment Tracking is turned off
Global seed set to 42
2023-06-25 18:21:59,953 - {pytorch_tabular.tabular_model:473} - INFO - Preparing the DataLoaders
2023-06-25 18:21:59,954 - {pytorch_tabular.tabular_datamodule:290} - INFO - Setting up the datamodule for classification task
2023-06-25 18:21:59,995 - {pytorch_tabular.tabular_model:521} - INFO - Preparing the Model: GatedAdditiveTreeEnsembleModel
2023-06-25 18:22:00,015 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
Auto select gpus: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
2023-06-25 18:22:00,071 - {pytorch_tabular.tabular_model:582} - INFO - Training Started


Using Data from: data\default-of-credit-card-clients for task: classification with 3 folds
{'dataset': 'default-of-credit-card-clients', 'batch_size': 512, 'max_epochs': 100, 'early_stopping_patience': 10, 'optimizer': 'AdamW', 'wandb': True, 'n_folds': 3}


c:\Users\manuj\miniconda3\envs\gate_skorch\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:604: UserWarning: Checkpoint directory C:\Work\Completed Research\gate_v3_experimentation\saved_models exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                       ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ GatedAdditiveTreesBackbone │  6.0 K │
│ 1 │ _embedding_layer │ Embedding1dLayer           │     46 │
│ 2 │ _head            │ Sequential                 │  1.3 K │
│ 3 │ loss             │ CrossEntropyLoss           │      0 │
└───┴──────────────────┴────────────────────────────┴────────┘

Trainable params: 7.3 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 7.3 K                                                                                                
Total estimated model params size (MB): 0

...

Output()

c:\Users\manuj\miniconda3\envs\gate_skorch\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\manuj\miniconda3\envs\gate_skorch\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


`Trainer.fit` stopped: `max_steps=1` reached.


2023-06-25 18:22:12,865 - {pytorch_tabular.tabular_model:584} - INFO - Training the model completed
2023-06-25 18:22:12,867 - {pytorch_tabular.tabular_model:1261} - INFO - Loading the best model
2023-06-25 18:22:12,868 - {pytorch_tabular.tabular_model:1268} - WARNING - No best model available to load. Did you run it more than 1 epoch?...


[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm1d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.


In [29]:
res_df = pd.DataFrame(res_l)
res_df

,dataset,model,macs,params,macs_h,params_h
0,default-of-credit-card-clients,GANDALF,7136.0,7278.0,7.136K,7.278K
1,heloc,GANDALF,39088.0,39952.0,39.088K,39.952K
2,eye_movements,GANDALF,9568.0,9734.0,9.568K,9.734K
3,Higgs,GANDALF,35968.0,36690.0,35.968K,36.690K
4,pol,GANDALF,13648.0,13880.0,13.648K,13.880K
5,albert,GANDALF,517024.0,520758.0,517.024K,520.758K
6,road-safety,GANDALF,141430.0,143417.0,141.430K,143.417K
7,MiniBooNE,GANDALF,212344.0,214394.0,212.344K,214.394K
8,covertype,GANDALF,366818.0,368801.0,366.818K,368.801K
9,jannis,GANDALF,37480.0,37746.0,37.480K,37.746K


In [30]:
res_df.to_parquet("output/gandalf_macs_params.parquet")

In [27]:
tabular_model = simulate_training_best_params(config)
mdl = tabular_model.model
mdl.eval()
tabular_model.datamodule.batch_size = 1
x = next(iter(tabular_model.datamodule.train_dataloader()))
macs, params = profile(mdl, inputs=(x,))
macs_h, params_h = clever_format([macs, params], "%.3f")

2023-06-25 18:18:12,002 - {pytorch_tabular.tabular_model:105} - INFO - Experiment Tracking is turned off
Global seed set to 42
2023-06-25 18:18:12,021 - {pytorch_tabular.tabular_model:473} - INFO - Preparing the DataLoaders
2023-06-25 18:18:12,024 - {pytorch_tabular.tabular_datamodule:290} - INFO - Setting up the datamodule for classification task
2023-06-25 18:18:12,037 - {pytorch_tabular.tabular_model:521} - INFO - Preparing the Model: GatedAdditiveTreeEnsembleModel
2023-06-25 18:18:12,055 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
Auto select gpus: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
2023-06-25 18:18:12,118 - {pytorch_tabular.tabular_model:582} - INFO - Training Started
c:\Users\manuj\miniconda3\envs\gate_skorch\lib\site-

Using Data from: data\heloc for task: classification with 3 folds
{'dataset': 'heloc', 'batch_size': 512, 'max_epochs': 100, 'early_stopping_patience': 10, 'optimizer': 'AdamW', 'wandb': True, 'n_folds': 3}


┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                       ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ GatedAdditiveTreesBackbone │ 38.9 K │
│ 1 │ _embedding_layer │ Embedding1dLayer           │     44 │
│ 2 │ _head            │ Sequential                 │  1.3 K │
│ 3 │ loss             │ CrossEntropyLoss           │      0 │
└───┴──────────────────┴────────────────────────────┴────────┘

Trainable params: 40.3 K                                                                                           
Non-trainable params: 0                                                                                            
Total params: 40.3 K                                                                                               
Total estimated model params size (MB): 0

Output()

c:\Users\manuj\miniconda3\envs\gate_skorch\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\manuj\miniconda3\envs\gate_skorch\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


`Trainer.fit` stopped: `max_steps=1` reached.


2023-06-25 18:18:12,283 - {pytorch_tabular.tabular_model:584} - INFO - Training the model completed
2023-06-25 18:18:12,283 - {pytorch_tabular.tabular_model:1261} - INFO - Loading the best model
2023-06-25 18:18:12,284 - {pytorch_tabular.tabular_model:1268} - WARNING - No best model available to load. Did you run it more than 1 epoch?...


In [17]:
tabular_model.datamodule.batch_size = 1
x = next(iter(tabular_model.datamodule.train_dataloader()))

In [24]:
macs, params = profile(mdl, inputs=(x,))
macs_h, params_h = clever_format([macs, params], "%.3f")

[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm1d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
